In [1]:
import json
import requests
import selenium
import undetected_chromedriver as uc
import time
import random
import pandas as pd



from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    StaleElementReferenceException,
    ElementClickInterceptedException
)

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC






scraped_data = []


headless = False

url = 'https://www.domiporta.pl/'

location_text = "Warszawa"

In [2]:
def human_like_typing(driver, element, text, min_delay=0.1, max_delay=0.3):
    actions = ActionChains(driver)
    element.click()  # Focus the input before typing
    for character in text:
        actions.send_keys(character).perform()
        time.sleep(random.uniform(min_delay, max_delay))

In [3]:

def scrape_current_page():
    # FIND HOUSE TAGS
    elements = driver.find_elements(By.CSS_SELECTOR, "article.sneakpeak")

    for idx, item in enumerate(elements):
        try:
            info_panel = item.find_element(By.CSS_SELECTOR, "div.sneakpeak__data")

            # Extract each piece of data safely
            title = info_panel.find_element(By.CSS_SELECTOR, "span.sneakpeak__title--bold").text.strip()
            address = info_panel.find_element(By.CSS_SELECTOR, "span.sneakpeak__title--inblock").text.strip()
            full_address = address.replace("dom Warszawa, ", "")
            price = info_panel.find_element(By.CSS_SELECTOR, "div.sneakpeak__price > span.sneakpeak__price_value").text.strip()
            area_house = info_panel.find_element(By.CSS_SELECTOR, "span.sneakpeak__details_item.sneakpeak__details_item--area").text.strip()
            area_land = info_panel.find_element(By.CSS_SELECTOR, "span.sneakpeak__details_item.sneakpeak__details_item--area-h").text.strip()
            price_per_m2 = info_panel.find_element(By.CSS_SELECTOR, "span.sneakpeak__details_item.sneakpeak__details_item--price").text.strip()
            
            # Correct handling of room elements
            room_elements = info_panel.find_elements(By.CSS_SELECTOR, "span.sneakpeak__details_item.sneakpeak__details_item--room")
            room = room_elements[0].text.strip() if room_elements else "N/A"

            #Link 
            link_element = info_panel.find_element(By.CSS_SELECTOR, "a.sneakpeak__title")
            link = link_element.get_attribute("href") if link_element else "N/A"

            # Output or store the data with correct indentation
            print(f"--- Result #{idx+1} ---")
            scraped_data.append({
    "Address": full_address,
    "Title": title,
    "Price": price,
    "Area": area_house,
    "Land Area": area_land,            
    "Rooms": room,
    "Price per m²": price_per_m2,
    "Link": link,
                })
            print()

        except Exception as e:
            print(f"⚠️ Error processing item #{idx+1}: {e}")




In [4]:
def scroll_to_bottom():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1.5)
    

In [5]:
def click_through_all_pages():
    page = 1
    while True:
        print(f"📄 Scraping page {page}...")

        # Scrape data from the current page
        scrape_current_page()

        # Scroll to the bottom to ensure pagination button is rendered
        scroll_to_bottom()

        try:
            # Wait for the next page button to be clickable
            next_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "li.pagination__link.pagination__link--right a"))
            )

            # Scroll button into view and attempt to click
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
            time.sleep(0.5)

            next_button.click()
        except ElementClickInterceptedException:
            print("⚠️ Click intercepted. Using JavaScript click as fallback.")
            driver.execute_script("arguments[0].click();", next_button)  # Fallback click
        except Exception as e:
            print(f"🚨 Error navigating to the next page: {e}")
            break  # Stop the loop if we encounter any other errors

        # Increment the page count and allow a brief pause between pages
        page += 1
        time.sleep(1.5) 


In [6]:
scraped_data = []

options = uc.ChromeOptions()
if headless:
    options.headless = True
options.add_argument("--start-maximized")
# options.add_argument(f"user-agent={self.get_random_user_agent()}")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")

driver = uc.Chrome(options=options, version_main=134) 
driver.get(url)

# Wait for and click the button with the specific id
accept_button = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
)
accept_button.click()

# Selecting "Dom" 
   
dropdown = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.ID, "CategoryName")))
dropdown.click()

dom_option = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "li.select-box__item[value='Dom']"))
)
dom_option.click()  # Click the "Dom" option

input_locator = (By.CSS_SELECTOR, 'input[placeholder="Wpisz lokalizację"]')

# Wait for the input to be clickable/interactable
input_field = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable(input_locator)
)

# Remove readonly attribute if necessary (some sites use readonly to block automation)
driver.execute_script("arguments[0].removeAttribute('readonly')", input_field)

# Call the human-like typing function
human_like_typing(driver, input_field, location_text)

input_field.send_keys(Keys.ENTER)

search_button_locator = (By.CSS_SELECTOR, "button.search__button.search__button--normal")

search_button_element = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable(search_button_locator))
search_button_element.click()

close_button_locator = (By.CSS_SELECTOR, "span.multiplication__bar_close.js-multiplication__bar_close")

close_button_element = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable(close_button_locator))
close_button_element.click()


click_through_all_pages()


df = pd.DataFrame(scraped_data)
df.to_csv("scraped_properties.csv", index=False)
print('DATA SAVED')

time.sleep(3)


📄 Scraping page 1...
--- Result #1 ---

--- Result #2 ---

--- Result #3 ---

--- Result #4 ---

--- Result #5 ---

--- Result #6 ---

--- Result #7 ---

--- Result #8 ---

--- Result #9 ---

--- Result #10 ---

--- Result #11 ---

--- Result #12 ---

--- Result #13 ---

--- Result #14 ---

--- Result #15 ---

--- Result #16 ---

--- Result #17 ---

--- Result #18 ---

--- Result #19 ---

--- Result #20 ---

--- Result #21 ---

--- Result #22 ---

--- Result #23 ---

⚠️ Error processing item #24: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.sneakpeak__details_item.sneakpeak__details_item--area-h"}
  (Session info: chrome=134.0.6998.166); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   undetected_chromedriver             0x0000000102916568 undetected_chromedriver + 6088040
1   undetected_chromedriver             0x000000010290e

In [7]:
import pandas as pd
from IPython.display import display

display(df)

,Address,Title,Price,Area,Land Area,Rooms,Price per m²,Link
0,"Wawer, Las, Sęczkowa",dom na sprzedaż 81m2,1 032 500 zł,"80,66 \nm\n2","0,0405 \nha",4,12 801 \nzł/m\n2,https://www.domiporta.pl/nieruchomosci/sprzeda...
1,"Wilanów, Zawady, Bruzdowa",dom na sprzedaż 224m2,2 890 000 zł,"223,90 \nm\n2","0,0600 \nha",7,12 908 \nzł/m\n2,https://www.domiporta.pl/nieruchomosci/sprzeda...
2,"Białołęka, Białołęka, Bohaterów",dom na sprzedaż 277m2,1 700 000 zł,277 \nm\n2,"0,0350 \nha",10,6 137 \nzł/m\n2,https://www.domiporta.pl/nieruchomosci/sprzeda...
3,Wilanów,dom na sprzedaż 300m2,3 400 000 zł,300 \nm\n2,"0,0750 \nha",6,11 333 \nzł/m\n2,https://www.domiporta.pl/nieruchomosci/sprzeda...
4,"Ursynów, Bekasów",dom na sprzedaż 293m2,3 490 000 zł,293 \nm\n2,"0,0423 \nha",5,11 911 \nzł/m\n2,https://www.domiporta.pl/nieruchomosci/sprzeda...
...,...,...,...,...,...,...,...,...
960,Mokotów,dom na sprzedaż 300m2,5 000 000 zł,300 \nm\n2,"0,0650 \nha",10,16 667 \nzł/m\n2,https://www.domiporta.pl/nieruchomosci/sprzeda...
961,Mokotów,dom na sprzedaż 500m2,11 000 000 zł,500 \nm\n2,"0,1300 \nha",11,22 000 \nzł/m\n2,https://www.domiporta.pl/nieruchomosci/sprzeda...
962,Ursynów,dom na sprzedaż 192m2,1 970 000 zł,192 \nm\n2,"0,0320 \nha",5,10 260 \nzł/m\n2,https://www.domiporta.pl/nieruchomosci/sprzeda...
963,"Ursus, Gołąbki",dom na sprzedaż 1050m2,4 990 000 zł,1050 \nm\n2,"0,0780 \nha",14,4 752 \nzł/m\n2,https://www.domiporta.pl/nieruchomosci/sprzeda...
